In [ ]:
# Pycache are evil, don't produce them
import sys
sys.dont_write_bytecode = True

import random

from portfolio_class import *

## Purpose of Strategies Functions
#### Return a dictionary with the structure: { [year, month] : [portfolio value, ROI (since the start of the portfolio)] }

In [ ]:
nasdaq_df = get_dataframe()

In [ ]:
def SP500(investment_date, divestment_date, dataframe):
    """
    DESCRIPTION:
        Calculate the monthly ROIs of a portfolio that invests in the 500 companies with the largest market cap.
        Be noted that the list of these 500 companies are dynamically calculated and changed monthly.
    
    INPUT SIGNATURE:
        1. investment_date (list): [year, month]
        2. divestment_date (list): [year, month]
        3. dataframe (Pandas dataframe): the source of our data, usually nasdaq_df

    OUTPUT SIGNATURE:
        1. monthly_ROIs (dictionary): { [year, month] : [portfolio value, ROI (since the start of the portfolio)] }
    """



# Helper Functions

In [ ]:
def the500(date, dataframe):
    """
    DESCRIPTION:
        Find the 500 companies with the largest market cap for the a given time (month within a year)

    INPUT SIGNATURE:
        1. date (list): [year, month]
        2. dataframe (Pandas dataframe): the source of our data, usually nasdaq_df

    OUTPUT SIGNATURE:
        1. largest_500 (list): a list contains tickers of those 500 companies sorted alphabetically
    """

    # filter out companies that we don't have information on volume of shares
    with_volume_df = dataframe.loc[dataframe["Volume"] > 0]

    # empty list to store tickers
    ticker_symbols = []

    # empty list to store corresponding market value of the tickers
    market_values = []

    # NOTE: from the 2 lists above, we will create a Pandas dataframe with 2 columns, and sort descending by the market value column

    # loop through the filtered dataframe and calculate each company's market value
    for row in range(len(with_volume_df)):

        ticker = with_volume_df.loc[row, "Ticker Symbol"]
        volume = with_volume_df.loc[row, "Volume"]
        close = with_volume_df.loc[row, "Close"] # we consider the closing price when finding out market cap

        market_value = close * volume

        # record the data
        ticker_symbols.append(ticker)
        market_values.append(market_value)

    # create a Pandas dataframe from the 2 lists to sort them
    data = {"Ticker" : ticker_symbols, "Market Value" : market_values}
    market_values_df = pd.DataFrame(data)

    # sort the dataframe
    market_values_df = market_values_df.sort_values(by = ["Market Value"], ascending = False)

    return market_values_df

In [ ]:
the500([2016, 12], nasdaq_df)